# Query-dependent link analysis

This notebook aims to identify the WUJ Start a Business pages using query-dependent link analysis

# Functions for coercing knowledge graph into NetworkX

In [ ]:
from neo4j import GraphDatabase
import networkx as nx
import os
import pandas as pd
from tqdm.notebook import tqdm
import operator
from operator import itemgetter
from collections import defaultdict
import gspread 
from oauth2client.service_account import ServiceAccountCredentials

In [ ]:
def getSubgraph(q, parameters=None):

    '''
    Given a Cypher query q, this function queries the knowledge graph,
    returns the nodes and edges from this query, and uses them to construct
    a networkx graph.

    E.g. getSubgraph(r'MATCH (u:Cid)-[r:HYPERLINKS_TO]->(v:Cid) RETURN *')
         returns the structural graph.

    Optionally, can add in parameters (dictionary), allowing Python variables
    to be integrated into the Cypher query q.

    E.g.
        parameters = {}
        parameters['pages'] = ['a','list','of','stuff']
        q7 = f"""
        MATCH (u:Cid)-[r]-(v:Cid)
        WHERE u.name IN $pages AND v.name in $pages
        RETURN *
        """

        g7 = getSubgraph(q7, parameters)
    '''

    # get credentials
    # add to .secrets: export KG_PWD="<PASSWORD>"
    KG_PWD = os.getenv("KG_PWD")

    # create connection to knowledge graph
    driver = GraphDatabase.driver(
        "bolt+s://knowledge-graph.integration.govuk.digital:7687",
        auth=("neo4j", KG_PWD),
    )

    # run query on knowledge graph
    results = driver.session().run(q, parameters)

    # create networkx graph object
    G = nx.MultiDiGraph()

    # add nodes into networkx graph object
    nodes = list(results.graph()._nodes.values())
    print("Adding nodes\n")
    for node in tqdm(nodes):
        G.add_node(node.id, labels=node._labels, properties=node._properties)

    # add edges into networkx graph object
    rels = list(results.graph()._relationships.values())
    print("Adding edges\n")
    for rel in tqdm(rels):
        G.add_edge(
            rel.start_node.id,
            rel.end_node.id,
            key=rel.id,
            type=rel.type,
            properties=rel._properties,
        )

    return G


def showGraph(g):
    """
    Given a networkx graph g, this function visualises the graph.
    Do not use for a large g.
    """
    print(nx.info(g))
    nx.draw(g)

In [ ]:
def getNoOfTruePages(g):
    """
    Calculate a proxy recall metric for the list of pages identified in a
    subgraph (when compared to the ground truth of the start a business pages). 
    The ouput is the number of pages in the subgraph list that are also in 
    the ground truth list.  
    """
    
    # convert nodeIds to page path slug for the subgraph list
    subgraph_list = [node[1]['properties']['name'] for node in g.nodes(data=True)]

    # set up the ground truth list
    true_list = list(sab_pages)

    # how many pages are in the subgraph list that are also in the ground truth list
    return 308 - len([node for node in true_list if node not in subgraph_list])
    

# Start a Business (pre-defined pages) graph

To investigate the graph of a manually curated list of a WUJ (Start a Business), i.e. the 'ideal' graph. Creates a graph of 308 nodes and 14295 edges

In [ ]:
GOOGLE_APPLICATION_CREDENTIALS = os.getenv('GOOGLE_APPLICATION_CREDENTIALS')

# Connect to service account
scope = ['https://spreadsheets.google.com/feeds'] 
credentials = ServiceAccountCredentials.from_json_keyfile_name(GOOGLE_APPLICATION_CREDENTIALS, scope) 
gc = gspread.authorize(credentials)

# Import the data from google sheets
spreadsheet_key = '1x3lHpUkIm-KTnWrRJU11An1J2Yb4KJT8z0QpuXOm3e0' 
book = gc.open_by_key(spreadsheet_key) 
worksheet = book.worksheet('sab_pages') 
table = worksheet.get_all_values()

# Convert table data into a dataframe then set 
df = pd.DataFrame(table[1:], columns=table[0])
sab_pages=set(df['pagePath'])

In [ ]:
# Query neo4j
parameters = {}
parameters['pages'] = list(sab_pages)

query = r"""
MATCH (c1:Cid)-[r:HYPERLINKS_TO|:USER_MOVEMENT]-(c2:Cid)
WHERE c1.name IN $pages
RETURN *
"""
g = getSubgraph(query, parameters)
showGraph(g)

In [ ]:
# Create a list of the nodes
nodes = list(g.nodes(data=True))

# Only keep nodes that are also in sab_pages
sabNodes = [node for node in nodes if node[1]['properties']['name'] in sab_pages]
g = g.subgraph([node[0] for node in sabNodes])

# Draw graph
showGraph(g)

## Exploring the Start a Business subgraph

Summary: 
- Subgraph components = 274, 16, 3, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1
- Max shortest path length = 17
- Centrality metrics provide an estimate of node importance, pages such as 
  '/get-information-about-a-company' and '/running-a-limited-company'
- Nodes with user movement between = 186
- User movement weight ranges from 6 to 34837

In [ ]:
## Which nodes are in the list but not in the graph? 
## Answer: two pages in the list no longer exist, so are not in the graph

# List of node property 'name'
nodes_graph = [v['properties']['name'] for _,v in g.nodes(data=True)]

# Which nodes are in sab_pages but not in the graph
[node for node in sab_pages if node not in nodes_graph]

In [ ]:
## Subgraph components: [274, 16, 3, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
nx.is_weakly_connected(g)
[len(c) for c in sorted(nx.weakly_connected_components(g), key=len, reverse=True)]

In [ ]:
list(nx.weakly_connected_components(g))

In [ ]:
## Max shortest path length: 17
max([max(j.values()) for (i,j) in nx.shortest_path_length(g)]) # find the largest diameter amongst all components within g

In [ ]:
# Between centrality: the number of shortest paths that pass through the node - a 'bridge' between nodes
g_di = nx.DiGraph(g)
g_di_between = nx.betweenness_centrality(g_di)
g_between = dict(sorted(g_di_between.items(), key=operator.itemgetter(1),reverse=True))


node_names = []
for k,v in g.nodes(data=True):
    node_names.append(k)
    node_names.append(v['properties']['name']) # append cid id and name    
node_names_dict = dict(zip(node_names[::2], node_names[1::2]))  # turn list into dict 
g_between = {v: node_names_dict.get(k, v) for k, v in g_between.items()}  # replace cid with node name 
list(g_between.items())[:10][:10]

In [ ]:
# Degree centrality: counts the number of incoming and outgoing relationships from a node - 'most connected'
g_degree = nx.degree_centrality(g)
g_degree = dict(sorted(g_degree.items(), key=operator.itemgetter(1),reverse=True))

[node[1]['properties']['name'] for node in G.nodes(data=True)]

node_names = []
for k,v in g.nodes(data=True):
    node_names.append(k)
    node_names.append(v['properties']['name'])  # append cid id and name    
node_names_dict = dict(zip(node_names[::2], node_names[1::2]))  # turn list into dict 
g_degree = {v: node_names_dict.get(k, v) for k, v in g_degree.items()}  # replace cid with node name 
list(g_degree.items())[:10][:10]

In [ ]:
# Closeness centrality: closeness to other nodes
g_closeness = nx.closeness_centrality(g)
g_closeness = dict(sorted(g_closeness.items(), key=operator.itemgetter(1),reverse=True))

node_names = []
for k,v in g.nodes(data=True):
    node_names.append(k)
    node_names.append(v['properties']['name'])  # append cid id and name    
node_names_dict = dict(zip(node_names[::2], node_names[1::2]))  # turn list into dict 
g_closeness = {v: node_names_dict.get(k, v) for k, v in g_closeness.items()}  # replace cid with node name 
list(g_closeness.items())[:10][:10]

In [ ]:
# Eigenvector centrality 
g_di = nx.DiGraph(g)
g_eigen = nx.eigenvector_centrality(g_di)
g_eigen = dict(sorted(g_eigen.items(), key=operator.itemgetter(1),reverse=True))

node_names = []
for k,v in g.nodes(data=True):
    node_names.append(k)
    node_names.append(v['properties']['name']) # append cid id and name    
node_names_dict = dict(zip(node_names[::2], node_names[1::2]))  # turn list into dict 
g_eigen = {v: node_names_dict.get(k, v) for k, v in g_eigen.items()}  # replace cid with node name 
list(g_eigen.items())[:10][:10]

In [ ]:
# VoteRank
g_voterank = nx.voterank(g)

node_names = []
for k,v in g.nodes(data=True):
    node_names.append(k)
    node_names.append(v['properties']['name']) # append cid id and name   
node_names_dict = dict(zip(node_names[::2], node_names[1::2]))  # turn list into dict 
g_voterank = [node_names_dict.get(item,item)  for item in g_voterank]
g_voterank[:10]

In [ ]:
# Which nodes have user_movement in or out? 186
user_movement_nodes = []

for node1, node2, data in g.edges(data=True):
    if data['type']=='USER_MOVEMENT':
            user_movement_nodes.append(node1)

len(list(set(user_movement_nodes)))

In [ ]:
# Weight (user movement) for each pair of nodes in the graph  
user_movement_weights = []

for node1, node2, data in g.edges(data=True):
    if data['type']=='USER_MOVEMENT':
        case = {'node1': node1, 'node2': node2, 'weight':data['properties']['weight'] }
        user_movement_weights.append(case)

sorted(user_movement_weights, key=itemgetter('weight'), reverse=True)

# Sum the weight of each node 
user_movements_sum = defaultdict(float)

for info in user_movement_weights:
    user_movements_sum[info['node1']] += info['weight']

user_movements_sum = [{'node1': node1, 'weight': user_movements_sum[node1]} 
                     for node1 in user_movements_sum]

sorted(user_movements_sum, key=lambda x: x['weight'], reverse=True)

# Defining subgraph based on session hits from BigQuery GA

In [ ]:
# Import df of nodes (contentId) that is based on at least 5 session hits in a two-week period 
df = pd.read_csv('..data/processed/content_ids.csv', index_col=0)

In [ ]:
# Graph
g = nx.read_gpickle("../data/processed/5_hits_per_contentID_graph.gpickle")

# Defining subgraphs to replicate the Start a Business pages

Exploring ways of creating a subgraph for a Start a Business WUJ (for query-dependent link analysis). 
<br>
Using the ground truth WUJ graph, and a managable subgraph (df).
<br><br>
Summary:
- Step by steps: 183 nodes, 27 in true list
- Browse pages: 173 nodes, 56 in true list
- Super-taxons: 2686 nodes, 154 in true list
- 'Top' nodes: 644 nodes, 137 in true list
- Keyword search: 370 nodes, 8 in true list
- Keyword search v2: 411 nodes, 11 in true list
- Merging all results: 4056 nodes, 191 in true list

### Using step by steps: 183 nodes,  27 in the true list (or with [r1:HYPERLINKS_TO|:USER_MOVEMENT]-(c3:Cid) 4209 nodes and 163 in true list)


In [ ]:
# Query neo4j
parameters = {}
parameters['contentId'] = df['contentId'].tolist()
parameters['pages'] = ['/export-customs-declaration', '/import-customs-declaration', '/import-goods-into-uk', 
                     '/set-up-limited-company', '/set-up-self-employed', '/export-goods'] 

query = f"""
MATCH (u:Cid)-[r:HYPERLINKS_TO|USER_MOVEMENT]->(v:Cid)
WHERE u.contentID IN $contentId AND v.contentID in $contentId 
WITH * 
MATCH (c1:Cid)-[r:HYPERLINKS_TO|:USER_MOVEMENT]-(c2:Cid)-[r1:HYPERLINKS_TO|:USER_MOVEMENT]-(c3:Cid)
WHERE c1.name IN $pages
RETURN *
"""

g = getSubgraph(query, parameters)
nx.info(g)

In [ ]:
# Query neo4j
parameters = {}
parameters['contentId'] = df['contentId'].tolist()
parameters['pages'] = ['/export-customs-declaration', '/import-customs-declaration', '/import-goods-into-uk', 
                       '/set-up-limited-company', '/set-up-self-employed', '/export-goods'] 

query = r"""
MATCH (u:Cid)-[r:HYPERLINKS_TO|USER_MOVEMENT]->(v:Cid)
WHERE u.contentID IN $contentId AND v.contentID in $contentId 
WITH *
MATCH (c1:Cid)
-[r1:HAS_STEP]->(s:Step)
-[r2:HAS_TASK]->(c2:Cid)
-[r3:HYPERLINKS_TO|USER_MOVEMENT]-(c3:Cid)
WHERE c1.name IN $pages
RETURN c2.name
"""

g = getSubgraph(query, parameters)
nx.info(g)

In [70]:
## Recall and precision

# Convert nodeIds to page path names
node_names_dict = {k:v['properties']['name'] for k,v in g.nodes(data=True)}

# Set up lists to compare
true_list = sab_pages
subgraph_list = list(node_names_dict.values())

# What pages are in the subgraph list but not in the true list? 
[node for node in subgraph_list if node not in true_list]

# What pages are in the true list but not in the subgraph list? 
[node for node in true_list if node not in subgraph_list]

# No. of pages in the subgraph list that are also in the true list
getNoOfTruePages(g)

49

### Using browse pages: 173 nodes, 56 in the true list ([r1:HYPERLINKS_TO|:USER_MOVEMENT]-(c3:Cid): 7157 nodes, 212 in true list)

In [ ]:
# Query neo4j
parameters = {}
parameters['contentId'] = df['contentId'].tolist()

query = f"""
MATCH (u:Cid)-[r:HYPERLINKS_TO|USER_MOVEMENT]->(v:Cid)
WHERE u.contentID IN $contentId AND v.contentID in $contentId
WITH * 
MATCH (c1:Cid)-[r:HYPERLINKS_TO|:USER_MOVEMENT]-(c2:Cid)
WHERE c1.name = '/browse/business'
RETURN *
"""

g1 = getSubgraph(query, parameters)
nx.info(g1)

In [ ]:
# No. of pages in the subgraph list that are also in the true list
getNoOfTruePages(g1)

### Using taxons - this is too big (over 58000 distinct nodes)

In [ ]:
# Query neo4j
parameters = {}
parameters['contentId'] = df['contentId'].tolist()

query = f"""
MATCH (u:Cid)-[r:HYPERLINKS_TO|USER_MOVEMENT]->(v:Cid)
WHERE u.contentID IN $contentId AND v.contentID in $contentId
WITH * 
MATCH (t:Taxon)<-[:IS_TAGGED_TO]-(c:Cid)
WHERE t.taxonBasePath STARTS WITH '/business'
RETURN *
"""

g2 = getSubgraph(query, parameters)
nx.info(g2)

In [ ]:
# No. of pages in the subgraph list that are also in the true list
getNoOfTruePages(g2)

### Using supertaxons: 2686 nodes, 154 in true list

In [ ]:
# Query neo4j
parameters = {}
parameters['contentId'] = df['contentId'].tolist()

query = f"""
MATCH (u:Cid)-[r:HYPERLINKS_TO|USER_MOVEMENT]->(v:Cid)
WHERE u.contentID IN $contentId AND v.contentID in $contentId
WITH * 
MATCH (t:Taxon)<-[:IS_TAGGED_TO]-(c:Cid)-[r:USER_MOVEMENT|:HYPERLINKS_TO]-(oc:Cid)-[:IS_TAGGED_TO]->(ot:Taxon)
WHERE t.taxonBasePath STARTS WITH '/business'
AND ot.taxonBasePath STARTS WITH '/money'
RETURN *
"""

g2 = getSubgraph(query, parameters)
nx.info(g2)

In [ ]:
# No. of pages in the subgraph list that are also in the true list
getNoOfTruePages(g2)

### Using the 'top' nodes (as defined by centrality metrics): 644 nodes, 137 in true list  ([r1:HYPERLINKS_TO|:USER_MOVEMENT]-(c3:Cid): 14306 nodes, 267 in true list)

In [ ]:
# Query neo4j
parameters = {}
parameters['contentId'] = df['contentId'].tolist()
parameters['pages'] = ['/get-information-about-a-company', '/set-up-business', '/search-for-trademark', '/pay-corporation-tax', 
         '/running-a-limited-company', '/limited-company-formation', '/business-support-helpline', '/capital-allowances',
         '/prepare-file-annual-accounts-for-limited-company', '/liquidate-your-company', '/set-up-sole-trader']

query = f"""
MATCH (u:Cid)-[r:HYPERLINKS_TO|USER_MOVEMENT]->(v:Cid)
WHERE u.contentID IN $contentId AND v.contentID in $contentId
WITH * 
MATCH (c1:Cid)-[r:HYPERLINKS_TO|:USER_MOVEMENT]-(c2:Cid)
WHERE c1.name IN $pages
RETURN *
"""

g3 = getSubgraph(query, parameters)
nx.info(g3)

In [ ]:
# No. of pages in the subgraph list that are also in the true list
getNoOfTruePages(g3)

### Using keyword search: 370 nodes, 8 in true list

In [ ]:
# Query neo4j
parameters = {}
parameters['contentId'] = df['contentId'].tolist()

query = f"""
MATCH (u:Cid)-[r:HYPERLINKS_TO|USER_MOVEMENT]->(v:Cid)
WHERE u.contentID IN $contentId AND v.contentID in $contentId
WITH * 
MATCH (c1:Cid)-[r:HYPERLINKS_TO|:USER_MOVEMENT]-(c2:Cid)
WHERE toLower(c1.text) CONTAINS 'start a business'
RETURN *
"""

g4 = getSubgraph(query, parameters)
nx.info(g4)

In [ ]:
# No. of pages in the subgraph list that are also in the true list
getNoOfTruePages(g4)

### Using keyword search v2: 411 nodes, 11 in true list

In [ ]:
# Query neo4j
parameters = {}
parameters['contentId'] = df['contentId'].tolist()

query = f"""
MATCH (u:Cid)-[r:HYPERLINKS_TO|USER_MOVEMENT]->(v:Cid)
WHERE u.contentID IN $contentId AND v.contentID in $contentId
WITH * 
MATCH (c1:Cid)-[r:HYPERLINKS_TO|:USER_MOVEMENT]-(c2:Cid)
WHERE toLower(c1.text) CONTAINS 'start business'
RETURN *
"""

g5 = getSubgraph(query, parameters)
nx.info(g5)

In [ ]:
# No. of pages in the subgraph list that are also in the true list
getNoOfTruePages(g5)

###  Merging results

In [ ]:
# Uncomment graphs to include, comment graphs to exclude
g_list = []
[g_list.append(v['properties']['name']) for k,v in g.nodes(data=True)]
[g_list.append(v['properties']['name']) for k,v in g1.nodes(data=True)]
[g_list.append(v['properties']['name']) for k,v in g2.nodes(data=True)]
[g_list.append(v['properties']['name']) for k,v in g3.nodes(data=True)]
#[g_list.append(v['properties']['name']) for k,v in g4.nodes(data=True)]
#[g_list.append(v['properties']['name']) for k,v in g5.nodes(data=True)]


In [ ]:
# Set up lists to compare
true_list = sab_pages
subgraph_list = g_list

# What pages are in the subgraph list but not in the true list? 
[node for node in subgraph_list if node not in true_list]

# What pages are in the true list but not in the subgraph list? 
a = [node for node in true_list if node not in subgraph_list]

# The no. of pages in the subgraph list that are also in the true list
308-len(a) 

In [ ]:
# Query neo4j to get a graph of the merged nodes
parameters = {}
parameters['pages'] = g_list

query = f"""
MATCH (u:Cid)-[r:HYPERLINKS_TO|USER_MOVEMENT]->(v:Cid)
WHERE u.name IN $pages AND v.name in $pages
RETURN *
"""

g = getSubgraph(query, parameters)
nx.info(g)

### One query filtering down

In [ ]:
# Query neo4j
parameters = {}
parameters['contentId'] = df['contentId'].tolist()
parameters['pages'] = ['/export-customs-declaration', '/import-customs-declaration', '/import-goods-into-uk', 
                       '/set-up-limited-company', '/set-up-self-employed', '/export-goods'] 

query = f"""
MATCH (u:Cid)-[r:HYPERLINKS_TO|USER_MOVEMENT]->(v:Cid)
WHERE u.contentID IN $contentId AND v.contentID in $contentId
WITH * 
MATCH (c1:Cid)-[r:HYPERLINKS_TO|:USER_MOVEMENT]-(c2:Cid)-[r1:HYPERLINKS_TO|:USER_MOVEMENT]-(c3:Cid)
WHERE c1.name = '/browse/business'
WITH *
MATCH (c4:Cid)-[r2:HYPERLINKS_TO|:USER_MOVEMENT]-(c5:Cid)
WHERE c4.name IN $pages
RETURN *
"""

g = getSubgraph(query, parameters)
nx.info(g)

In [ ]:
# No. of pages in the subgraph list that are also in the true list
getNoOfTruePages(g)

###  Apply PageRank

In [ ]:
# PageRank computes a ranking of the nodes in the graph G based on the structure of the incoming links
# weightkey: Edge data key to use as weight. If None weights are set to 1.
pagerank = nx.pagerank(g, alpha=0.85, personalization=None, max_iter=100, tol=1e-06, nstart=None, weight='weight', dangling=None)

# Set PageRank in descending order
pagerank = dict(sorted(pagerank.items(), key=lambda item: item[1], reverse=True))

# Convert cid id to name 
node_names_dict = {k:v['properties']['name'] for k,v in g.nodes(data=True)}
pagerank_sorted = {v: node_names_dict.get(k, v) for k, v in pagerank.items()}  # replace cid with node name 
pagerank_sorted


In [ ]:
# Where in the PageRank list do the pre-defined pages sit? 
all_pagepaths = list(pagerank_sorted.values())

for pagepath1 in all_pagepaths:
    for pagepath2 in sab_pages:
        if pagepath1 == pagepath2: 
           print(all_pagepaths.index(pagepath1))

In [ ]:
# Export PageRank to csv
df = pd.DataFrame.from_dict(pagerank_sorted, orient="index")
df.to_csv("../data/processed/sab_wuj_pagerank.csv")